# Iris Dataset

## Importing algorithm

In [1]:
import numpy as np 
import pandas as pd
from ipynb.fs.full.part_2_algorithm import QCBA, QCBATransformation, RuleTrimmer, RuleRefitter, RuleLiteralPruner, RulePostPruner, RuleOverlapPruner, RuleExtender, QuantitativeDataFrame, LiteralCache, Interval, IntervalReader, QuantitativeCAR  
from ipynb.fs.full.part_2_algorithm import TransactionDB, CBA, M1Algorithm, top_rules, createCARs, generateCARs, Pruning 

In [2]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

## Reading dataset

In [3]:
df = pd.read_table('../datasets/iris.data', sep = ',', header = None) #C:/Users/jeral/OneDrive/Desktop/DM/datasets/

In [4]:
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Cleaning dataset

In [5]:
df.rename(columns={0: 'Sepal Length', 1: 'Sepal Width', 2: 'Petal Length', 3: 'Petal Width', 4: 'Class'}, inplace = True)

In [6]:
df.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Train test split 

In [7]:
X, y = make_blobs(n_samples=df.shape[0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(120, 2) (30, 2) (120,) (30,)


In [8]:
train = df.sample(n=X_train.shape[0])
test = pd.concat([train,df]).drop_duplicates(keep=False)
train.shape, test.shape

((120, 5), (29, 5))

In [9]:
txns_train = TransactionDB.from_DataFrame(train)
txns_test = TransactionDB.from_DataFrame(test)

In [10]:
print(train.head())
train.shape

     Sepal Length  Sepal Width  Petal Length  Petal Width           Class
0             5.1          3.5           1.4          0.2     Iris-setosa
136           6.3          3.4           5.6          2.4  Iris-virginica
144           6.7          3.3           5.7          2.5  Iris-virginica
121           5.6          2.8           4.9          2.0  Iris-virginica
137           6.4          3.1           5.5          1.8  Iris-virginica


(120, 5)

In [11]:
print(test.head())
test.shape

    Sepal Length  Sepal Width  Petal Length  Petal Width        Class
12           4.8          3.0           1.4          0.1  Iris-setosa
14           5.8          4.0           1.2          0.2  Iris-setosa
16           5.4          3.9           1.3          0.4  Iris-setosa
18           5.7          3.8           1.7          0.3  Iris-setosa
25           5.0          3.0           1.6          0.2  Iris-setosa


(29, 5)

## Mining Class Association Rules

### Finding all rules

In [12]:
cars = generateCARs(txns_train, support = 2.51, confidence = 1)

In [13]:
print("Rule: " , cars)

Rule:  [CAR {Petal Width=0.2} => {Class=Iris-setosa} sup: 0.19 conf: 1.00 id: 53
, CAR {Petal Length=1.5} => {Class=Iris-setosa} sup: 0.09 conf: 1.00 id: 48
, CAR {Petal Width=1.3} => {Class=Iris-versicolor} sup: 0.08 conf: 1.00 id: 42
, CAR {Petal Length=1.4} => {Class=Iris-setosa} sup: 0.07 conf: 1.00 id: 40
, CAR {Petal Length=4.5} => {Class=Iris-versicolor} sup: 0.06 conf: 1.00 id: 32
, CAR {Petal Length=1.4,Petal Width=0.2} => {Class=Iris-setosa} sup: 0.06 conf: 1.00 id: 39
, CAR {Petal Width=1.0} => {Class=Iris-versicolor} sup: 0.05 conf: 1.00 id: 16
, CAR {Petal Length=1.3} => {Class=Iris-setosa} sup: 0.05 conf: 1.00 id: 21
, CAR {Petal Length=5.6} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 23
, CAR {Petal Width=2.0} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 25
, CAR {Petal Width=2.3} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 26
, CAR {Sepal Width=3.4,Petal Width=0.2} => {Class=Iris-setosa} sup: 0.05 conf: 1.00 id: 43
, CAR {Petal Length=1.6} => {Class=

In [14]:
print("There are {} Relation derived.".format(len(cars)))

There are 57 Relation derived.


In [15]:
cars

[CAR {Petal Width=0.2} => {Class=Iris-setosa} sup: 0.19 conf: 1.00 id: 53,
 CAR {Petal Length=1.5} => {Class=Iris-setosa} sup: 0.09 conf: 1.00 id: 48,
 CAR {Petal Width=1.3} => {Class=Iris-versicolor} sup: 0.08 conf: 1.00 id: 42,
 CAR {Petal Length=1.4} => {Class=Iris-setosa} sup: 0.07 conf: 1.00 id: 40,
 CAR {Petal Length=4.5} => {Class=Iris-versicolor} sup: 0.06 conf: 1.00 id: 32,
 CAR {Petal Length=1.4,Petal Width=0.2} => {Class=Iris-setosa} sup: 0.06 conf: 1.00 id: 39,
 CAR {Petal Width=1.0} => {Class=Iris-versicolor} sup: 0.05 conf: 1.00 id: 16,
 CAR {Petal Length=1.3} => {Class=Iris-setosa} sup: 0.05 conf: 1.00 id: 21,
 CAR {Petal Length=5.6} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 23,
 CAR {Petal Width=2.0} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 25,
 CAR {Petal Width=2.3} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 26,
 CAR {Sepal Width=3.4,Petal Width=0.2} => {Class=Iris-setosa} sup: 0.05 conf: 1.00 id: 43,
 CAR {Petal Length=1.6} => {Class=Iris-se

### Finding the top rules

In [18]:
rules = top_rules(txns_train.string_representation)
index = 0
while True:
    print("Rule: ", rules[index])
    print("\n")
    index = index + 1
    if index == len(rules):
        break

Rule:  ('Class:=:Iris-setosa', ('Petal Width:=:0.2',), 0.19166666666666668, 1.0)


Rule:  ('Petal Width:=:0.2', ('Class:=:Iris-setosa',), 0.19166666666666668, 0.6052631578947368)


Rule:  ('Class:=:Iris-versicolor', ('Sepal Width:=:3.0',), 0.06666666666666667, 0.42105263157894735)


Rule:  ('Sepal Width:=:3.0', ('Class:=:Iris-versicolor',), 0.06666666666666667, 0.18604651162790697)


Rule:  ('Class:=:Iris-virginica', ('Sepal Width:=:3.0',), 0.06666666666666667, 0.42105263157894735)


Rule:  ('Sepal Width:=:3.0', ('Class:=:Iris-virginica',), 0.06666666666666667, 0.20512820512820512)


Rule:  ('Class:=:Iris-setosa', ('Sepal Width:=:3.0',), 0.025, 0.15789473684210525)


Rule:  ('Sepal Width:=:3.0', ('Class:=:Iris-setosa',), 0.025, 0.07894736842105263)


Rule:  ('Class:=:Iris-setosa', ('Sepal Width:=:3.0', 'Petal Width:=:0.2'), 0.016666666666666666, 1.0)


Rule:  ('Petal Width:=:0.2', ('Sepal Width:=:3.0', 'Class:=:Iris-setosa'), 0.016666666666666666, 0.6666666666666666)


Rule:  ('Sepal W

Rule:  ('Class:=:Iris-versicolor', ('Petal Width:=:1.4', 'Sepal Width:=:2.9'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Width:=:2.9', ('Petal Width:=:1.4', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.14285714285714285)


Rule:  ('Petal Width:=:1.4', ('Sepal Width:=:2.9', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.16666666666666666)


Rule:  ('Sepal Width:=:2.9', ('Petal Width:=:1.4',), 0.008333333333333333, 0.125)


Rule:  ('Petal Width:=:1.4', ('Sepal Width:=:2.9',), 0.008333333333333333, 0.1111111111111111)


Rule:  ('Class:=:Iris-versicolor', ('Sepal Length:=:5.1',), 0.008333333333333333, 0.125)


Rule:  ('Sepal Length:=:5.1', ('Class:=:Iris-versicolor',), 0.008333333333333333, 0.023255813953488372)


Rule:  ('Class:=:Iris-setosa', ('Sepal Length:=:5.1',), 0.058333333333333334, 0.875)


Rule:  ('Sepal Length:=:5.1', ('Class:=:Iris-setosa',), 0.058333333333333334, 0.18421052631578946)


Rule:  ('Class:=:Iris-setosa', ('Sepal Length:=:5.1', 'Petal Width:=:0.2'), 0.

Rule:  ('Sepal Width:=:2.7', ('Petal Length:=:5.1', 'Class:=:Iris-versicolor'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:5.1', ('Sepal Width:=:2.7', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.3333333333333333)


Rule:  ('Class:=:Iris-virginica', ('Petal Length:=:5.1', 'Sepal Width:=:2.7'), 0.016666666666666666, 0.6666666666666666)


Rule:  ('Sepal Width:=:2.7', ('Petal Length:=:5.1', 'Class:=:Iris-virginica'), 0.016666666666666666, 0.3333333333333333)


Rule:  ('Petal Length:=:5.1', ('Sepal Width:=:2.7', 'Class:=:Iris-virginica'), 0.016666666666666666, 0.5)


Rule:  ('Sepal Width:=:2.7', ('Petal Length:=:5.1',), 0.025, 0.42857142857142855)


Rule:  ('Petal Length:=:5.1', ('Sepal Width:=:2.7',), 0.025, 0.42857142857142855)


Rule:  ('Class:=:Iris-virginica', ('Petal Width:=:2.0',), 0.05, 1.0)


Rule:  ('Petal Width:=:2.0', ('Class:=:Iris-virginica',), 0.05, 0.15384615384615385)


Rule:  ('Class:=:Iris-virginica', ('Petal Width:=:2.0', 'Sepal Width:=:3.0'), 0.0083333

Rule:  ('Petal Width:=:1.3', ('Sepal Length:=:5.5', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.25)


Rule:  ('Sepal Length:=:5.5', ('Petal Width:=:1.3', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.1)


Rule:  ('Petal Width:=:1.3', ('Sepal Length:=:5.5',), 0.008333333333333333, 0.16666666666666666)


Rule:  ('Sepal Length:=:5.5', ('Petal Width:=:1.3',), 0.008333333333333333, 0.1)


Rule:  ('Class:=:Iris-setosa', ('Sepal Length:=:5.5', 'Petal Length:=:1.4'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:1.4', ('Sepal Length:=:5.5', 'Class:=:Iris-setosa'), 0.008333333333333333, 0.5)


Rule:  ('Sepal Length:=:5.5', ('Petal Length:=:1.4', 'Class:=:Iris-setosa'), 0.008333333333333333, 0.1111111111111111)


Rule:  ('Petal Width:=:0.2', ('Sepal Length:=:5.5', 'Petal Length:=:1.4'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:1.4', ('Sepal Length:=:5.5', 'Petal Width:=:0.2'), 0.008333333333333333, 0.5)


Rule:  ('Sepal Length:=:5.5', ('Petal Length:=:1.4', 'Peta

Rule:  ('Class:=:Iris-versicolor', ('Sepal Length:=:6.1', 'Sepal Width:=:2.9'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Width:=:2.9', ('Sepal Length:=:6.1', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.25)


Rule:  ('Sepal Length:=:6.1', ('Sepal Width:=:2.9', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.16666666666666666)


Rule:  ('Sepal Width:=:2.9', ('Sepal Length:=:6.1',), 0.008333333333333333, 0.16666666666666666)


Rule:  ('Sepal Length:=:6.1', ('Sepal Width:=:2.9',), 0.008333333333333333, 0.1111111111111111)


Rule:  ('Class:=:Iris-versicolor', ('Sepal Length:=:6.1', 'Petal Width:=:1.4'), 0.016666666666666666, 0.6666666666666666)


Rule:  ('Petal Width:=:1.4', ('Sepal Length:=:6.1', 'Class:=:Iris-versicolor'), 0.016666666666666666, 0.5)


Rule:  ('Sepal Length:=:6.1', ('Petal Width:=:1.4', 'Class:=:Iris-versicolor'), 0.016666666666666666, 0.2857142857142857)


Rule:  ('Class:=:Iris-virginica', ('Sepal Length:=:6.1', 'Petal Width:=:1.4'), 0.008333333333333333, 0

...

Rule:  ('Sepal Length:=:7.4', ('Petal Length:=:6.1', 'Class:=:Iris-virginica'), 0.008333333333333333, 0.3333333333333333)


Rule:  ('Sepal Width:=:2.8', ('Sepal Length:=:7.4', 'Petal Length:=:6.1'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:6.1', ('Sepal Length:=:7.4', 'Sepal Width:=:2.8'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:7.4', ('Petal Length:=:6.1', 'Sepal Width:=:2.8'), 0.008333333333333333, 1.0)


Rule:  ('Petal Width:=:1.9', ('Sepal Length:=:7.4', 'Petal Length:=:6.1'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:6.1', ('Sepal Length:=:7.4', 'Petal Width:=:1.9'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:7.4', ('Petal Length:=:6.1', 'Petal Width:=:1.9'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:6.1', ('Sepal Length:=:7.4',), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:7.4', ('Petal Length:=:6.1',), 0.008333333333333333, 0.3333333333333333)


Rule:  ('Class:=:Iris-virginica', ('Sepal Length:=:7.9',), 0.00833333

Rule:  ('Petal Width:=:1.9', ('Petal Length:=:5.3', 'Sepal Width:=:2.7'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:5.3', ('Petal Width:=:1.9', 'Sepal Width:=:2.7'), 0.008333333333333333, 0.3333333333333333)


Rule:  ('Sepal Length:=:6.4', ('Petal Length:=:5.3', 'Petal Width:=:1.9'), 0.008333333333333333, 1.0)


Rule:  ('Petal Width:=:1.9', ('Petal Length:=:5.3', 'Sepal Length:=:6.4'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:5.3', ('Petal Width:=:1.9', 'Sepal Length:=:6.4'), 0.008333333333333333, 1.0)


Rule:  ('Petal Width:=:1.9', ('Petal Length:=:5.3',), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:5.3', ('Petal Width:=:1.9',), 0.008333333333333333, 0.25)


Rule:  ('Class:=:Iris-virginica', ('Sepal Length:=:7.3',), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:7.3', ('Class:=:Iris-virginica',), 0.008333333333333333, 0.02564102564102564)


Rule:  ('Class:=:Iris-virginica', ('Sepal Length:=:7.3', 'Petal Width:=:1.8'), 0.008333333333333333, 1.0)




Rule:  ('Sepal Length:=:4.5', ('Class:=:Iris-setosa',), 0.008333333333333333, 0.02631578947368421)


Rule:  ('Class:=:Iris-setosa', ('Sepal Length:=:4.5', 'Petal Length:=:1.3'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:1.3', ('Sepal Length:=:4.5', 'Class:=:Iris-setosa'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:4.5', ('Petal Length:=:1.3', 'Class:=:Iris-setosa'), 0.008333333333333333, 0.16666666666666666)


Rule:  ('Petal Length:=:1.3', ('Sepal Length:=:4.5',), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:4.5', ('Petal Length:=:1.3',), 0.008333333333333333, 0.16666666666666666)


Rule:  ('Class:=:Iris-setosa', ('Sepal Length:=:4.5', 'Petal Width:=:0.3'), 0.008333333333333333, 1.0)


Rule:  ('Petal Width:=:0.3', ('Sepal Length:=:4.5', 'Class:=:Iris-setosa'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:4.5', ('Petal Width:=:0.3', 'Class:=:Iris-setosa'), 0.008333333333333333, 0.2)


Rule:  ('Petal Length:=:1.3', ('Sepal Length:=:4.5', 'Petal Width:

Rule:  ('Sepal Length:=:6.8', ('Petal Length:=:4.8',), 0.008333333333333333, 0.3333333333333333)


Rule:  ('Class:=:Iris-virginica', ('Petal Length:=:6.6',), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:6.6', ('Class:=:Iris-virginica',), 0.008333333333333333, 0.02564102564102564)


Rule:  ('Class:=:Iris-virginica', ('Petal Length:=:6.6', 'Sepal Width:=:3.0'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Width:=:3.0', ('Petal Length:=:6.6', 'Class:=:Iris-virginica'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:6.6', ('Sepal Width:=:3.0', 'Class:=:Iris-virginica'), 0.008333333333333333, 0.125)


Rule:  ('Sepal Width:=:3.0', ('Petal Length:=:6.6',), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:6.6', ('Sepal Width:=:3.0',), 0.008333333333333333, 0.05263157894736842)


Rule:  ('Class:=:Iris-virginica', ('Petal Length:=:6.6', 'Petal Width:=:2.1'), 0.008333333333333333, 1.0)


Rule:  ('Petal Width:=:2.1', ('Petal Length:=:6.6', 'Class:=:Iris-virginica'), 0.008333333333

Rule:  ('Sepal Width:=:2.0', ('Petal Length:=:3.5',), 0.008333333333333333, 0.5)


Rule:  ('Class:=:Iris-versicolor', ('Petal Length:=:3.8',), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:3.8', ('Class:=:Iris-versicolor',), 0.008333333333333333, 0.023255813953488372)


Rule:  ('Class:=:Iris-versicolor', ('Petal Length:=:3.8', 'Sepal Length:=:5.5'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Length:=:5.5', ('Petal Length:=:3.8', 'Class:=:Iris-versicolor'), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:3.8', ('Sepal Length:=:5.5', 'Class:=:Iris-versicolor'), 0.008333333333333333, 0.25)


Rule:  ('Sepal Length:=:5.5', ('Petal Length:=:3.8',), 0.008333333333333333, 1.0)


Rule:  ('Petal Length:=:3.8', ('Sepal Length:=:5.5',), 0.008333333333333333, 0.16666666666666666)


Rule:  ('Class:=:Iris-versicolor', ('Petal Length:=:3.8', 'Sepal Width:=:2.4'), 0.008333333333333333, 1.0)


Rule:  ('Sepal Width:=:2.4', ('Petal Length:=:3.8', 'Class:=:Iris-versicolor'), 0.0083333333333333

In [19]:
print("Total number of top rules:", len(rules))

Total number of top rules: 4562


## Building a classifier

In [20]:
cba = CBA()
classifier = M1Algorithm(cars, txns_train).build()

In [21]:
classifier.rules

[CAR {Petal Width=0.2} => {Class=Iris-setosa} sup: 0.19 conf: 1.00 id: 53,
 CAR {Petal Length=1.5} => {Class=Iris-setosa} sup: 0.09 conf: 1.00 id: 48,
 CAR {Petal Width=1.3} => {Class=Iris-versicolor} sup: 0.08 conf: 1.00 id: 42,
 CAR {Petal Length=1.4} => {Class=Iris-setosa} sup: 0.07 conf: 1.00 id: 40,
 CAR {Petal Length=4.5} => {Class=Iris-versicolor} sup: 0.06 conf: 1.00 id: 32,
 CAR {Petal Width=1.0} => {Class=Iris-versicolor} sup: 0.05 conf: 1.00 id: 16,
 CAR {Petal Length=1.3} => {Class=Iris-setosa} sup: 0.05 conf: 1.00 id: 21,
 CAR {Petal Length=5.6} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 23,
 CAR {Petal Width=2.0} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 25,
 CAR {Petal Width=2.3} => {Class=Iris-virginica} sup: 0.05 conf: 1.00 id: 26,
 CAR {Petal Length=1.6} => {Class=Iris-setosa} sup: 0.04 conf: 1.00 id: 9,
 CAR {Petal Width=1.2} => {Class=Iris-versicolor} sup: 0.04 conf: 1.00 id: 10,
 CAR {Petal Length=4.7} => {Class=Iris-versicolor} sup: 0.04 conf: 1.0

In [22]:
print(len(classifier.rules))

25


In [23]:
accuracy = classifier.test_transactions(txns_test)
print("Classifier accuracy:", accuracy)

Classifier accuracy: 0.896551724137931


## Optimization

In [24]:
quant_dataframe_train_disc = QuantitativeDataFrame(df)

In [25]:
cba = CBA()
cba.fit(txns_train)

In [26]:
qcba_cba = QCBA(quant_dataframe_train_disc, cba_rule_model=cba)

In [27]:
refitting=True
literal_pruning=True
trimming=True
extension=True
overlap_pruning=True
QCBA = qcba_cba.param(refitting,literal_pruning,trimming,extension,overlap_pruning, accuracy)

In [28]:
qcba_cba.fit()

applying selected transformations
refitting
literal pruning
trimming
extending
[                                                  ]
post pruning
overlap pruning


In [29]:
print("CBA accuracy:", accuracy)
print("QCBA accuracy:", qcba_cba.score(QCBA, quant_dataframe_train_disc,txns_train))

CBA accuracy: 0.896551724137931
QCBA accuracy: 0.99805
